In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp pctexamples

In [ ]:
#| export
from pct.hierarchy import PCTHierarchy
from pct.environment_processing import EnvironmentProcessingFactory

## PCTExamples Class

The `PCTExamples` class is designed to load a PCTHierarchy configuration file and provide various methods to interact with the hierarchy.

In [ ]:
#| export
class PCTExamples:
    """
    PCTExamples class provides methods to load a PCT hierarchy from a configuration file, 
    summarize the hierarchy, get the configuration, draw the hierarchy, run the hierarchy 
    for a specified number of steps, and close the environment.
    Attributes:
        hierarchy (PCTHierarchy): The loaded PCT hierarchy.
        env (Environment): The environment associated with the PCT hierarchy.
    """

    def __init__(self, config_file, min=True, early_termination=False, history=False, additional_props=None, render = False):
        """
        Initializes the PCTExamples instance by loading the hierarchy from the given configuration file.
        """
        self.config_file = config_file
        self.hierarchy, self.env, self.environment_properties = PCTHierarchy.load_from_file(config_file, min=min, early_termination=early_termination, 
                                                                                            history=history, additional_props=additional_props, render=render)
        self.history_data = None
        self.history = history

    def summary(self):
        """
        Prints a summary of the hierarchy.
        """
        self.hierarchy.summary()

    def get_config(self):
        """
        Returns the configuration of the hierarchy.
        """
        return self.hierarchy.get_config()

    def draw(self, with_labels=True, with_edge_labels=False, font_size=12, font_weight='bold', font_color='black', 
                       color_mapping={'PL':'aqua','OL':'limegreen','CL':'goldenrod', 'RL':'red', 'I':'silver', 'A':'yellow'},
                       node_size=500, arrowsize=25, align='horizontal', file=None, figsize=(8,8), move={}, draw_fig=True,
                       node_color=None, layout={'r':2,'c':1,'p':2, 'o':0}, funcdata=False, interactive_mode=False, experiment=None):
        """
        Draws the hierarchy with the specified parameters and returns the figure.
        """
        fig = self.hierarchy.draw(with_labels, with_edge_labels, font_size, font_weight, font_color, color_mapping, node_size, 
                            arrowsize, align, file, figsize, move, draw_fig, node_color, layout, funcdata, interactive_mode, experiment)
        
        return fig

    def run(self, steps=1, verbose=False):#, render=False):
        """
        Runs the hierarchy for the specified number of steps and returns the result.
        """
        # self.hierarchy.get_preprocessor()[0].set_render(render)
        return self.hierarchy.run(steps, verbose)

    def results(self, verbose=False, args=None, environment_properties=None):
        print(self.environment_properties)
        env_name = self.env.get_name()
        env_proc = EnvironmentProcessingFactory.createEnvironmentProcessing(f'{env_name}EnvironmentProcessing')
        allargs={'drive':'', 'verbosed':{'hpct_verbose':verbose}, 'max': False} | args
        env_proc.set_properties(allargs)
        env_proc.results(filepath=self.config_file, environment_properties=self.environment_properties|environment_properties, hierarchy=self.hierarchy)


    def close(self):
        """
        Closes the environment.
        """
        self.env.close()

    def get_history_keys(self):
        """
        Returns the keys of the history data of the hierarchy.
        """
        if self.history :
            if self.history_data is None:
                self.history_data = self.hierarchy.get_history_data()
            return list(self.history_data.keys())
        else:
            print("History is not enabled")

        return None

    def set_history_data(self):
        """
        Sets the history data of the hierarchy.
        """
        if self.history :

            if self.history_data is None:
                self.history_data = self.hierarchy.get_history_data()

            return self.history_data
        else:
            print("History is not enabled")

        return None

    def get_config_file_contents(self):
        """
        Reads and returns the contents of the config file.
        """
        with open(self.config_file, 'r') as file:
            return file.read()

    def plot_history(self, plots=None, title_prefix='', plots_dir=None, plots_figsize=(12, 6), history_data=None):
        """
        Plots the history of the hierarchy.
        """
        print(plots)
        plots = self.hierarchy.get_plots_config(plots, title_prefix)
        print(plots)

        from os import sep
        for plot in plots:
            print(plot)
            plotfile=None
            if plots_dir:
                plotfile = plots_dir + sep + plot['title'] + '-' + str(self.hierarchy.get_namespace()) + '.png'
            fig = self.hierarchy.hierarchy_plots(title=plot['title'], plot_items=plot['plot_items'], figsize=plots_figsize, file=plotfile, history=history_data)
            # import matplotlib.pyplot as plt
            # plt.close(fig)  # Close the figure here
    
        return fig
    
    def plot_single(self, plot=None, title_prefix='', plots_dir=None, plots_figsize=(12, 6), history_data=None):
        """
        Plot one item of the history of the hierarchy.
        """

        from os import sep, path, makedirs

        plotfile=None
        if plots_dir:
            if not path.exists(plots_dir):
                makedirs(plots_dir)
            plotfile = plots_dir + sep + title_prefix + plot['title'] + '-' + str(self.hierarchy.get_namespace()) + '.png'
        fig = self.hierarchy.hierarchy_plots(title=title_prefix + plot['title'], plot_items=plot['plot_items'], figsize=plots_figsize, file=plotfile, history=history_data)
            # import matplotlib.pyplot as plt
            # plt.close(fig)  # Close the figure here
    
        return fig
    
    @classmethod
    def run_example(cls, config_file, 
                    run_hierarchy=True,
                    render=False, 
                    create_image=False,
                    create_video=False,
                    early_termination=False,
                    steps=None,
                    print_summary=False,
                    return_config=False,
                    display_usage=False,
                    create_plots=False,
                    verbose=False,
                    image_params=None,
                    video_params=None,
                    plot_params=None,
                    history=None,  # Allow explicit history control
                    **kwargs):
        """
        Class method to run a PCT hierarchy example with various options.
        
        Args:
            config_file (str): Path to the configuration file
            run_hierarchy (bool): Whether to run the hierarchy (default: True)
            render (bool): Whether to render the environment (default: False)
            create_image (bool): Whether to create hierarchy image (default: False)
            create_video (bool): Whether to create video file (default: False)
            early_termination (bool): Whether to enable early termination (default: False)
            steps (int): Number of steps to run (overrides config if specified)
            print_summary (bool): Whether to print hierarchy summary (default: False)
            return_config (bool): Whether to return the configuration (default: False)
            display_usage (bool): Whether to display usage information (default: False)
            create_plots (bool): Whether to create and return plots (default: False)
            verbose (bool): Whether to enable verbose output (default: False)
            image_params (dict): Parameters for image creation
            video_params (dict): Parameters for video creation
            plot_params (dict): Parameters for plot creation
            history (bool): Explicit control over history tracking (default: None - auto-determined)
            **kwargs: Additional parameters passed to PCTExamples constructor
        
        Returns:
            dict: Results containing requested outputs and run results
        """
        
        if display_usage:
            usage_text = """
            PCTExamples.run_example() Usage:
            
            Basic usage:
                PCTExamples.run_example('testfiles/MountainCar/MountainCar-cdf7cc1497ad143c0b04a3d9e72ab783.properties')
            
            With options:
                PCTExamples.run_example(
                    config_file='testfiles/MountainCar/MountainCar-cdf7cc1497ad143c0b04a3d9e72ab783.properties',
                    run_hierarchy=True,      # Run the hierarchy
                    render=True,             # Render environment
                    create_image=True,       # Create hierarchy diagram
                    create_video=True,       # Create video file
                    create_plots=True,       # Create and return plots
                    early_termination=True,  # Enable early termination
                    steps=1000,              # Override step count
                    print_summary=True,      # Print hierarchy summary
                    return_config=True,      # Return configuration
                    verbose=True             # Verbose output
                )
            
            Returns dictionary with results and any requested outputs.
            """
            print(usage_text)
            if not any([run_hierarchy, create_image, create_video, create_plots, print_summary, return_config]):
                return {"usage_displayed": True}
        
        # Initialize default parameters
        if image_params is None:
            image_params = {
                'with_labels': True,
                'figsize': (12, 8),
                'file': None
            }
        
        if video_params is None:
            video_params = {
                'fps': 30,
                'filename': None
            }
        
        if plot_params is None:
            plot_params = {
                'plots': None,
                'title_prefix': '',
                'plots_dir': '/tmp/plots',
                'plots_figsize': (12, 6),
                'single_plot': False
            }
        
        results = {}
        
        try:
            # History is only needed for plots, not videos
            # Video is controlled separately by hierarchy video property
            if history is None:
                # Auto-determine: Only enable history if plots are requested
                enable_history = create_plots
            else:
                # Use explicit user setting
                enable_history = history
            
            # Create PCTExamples instance
            example = cls(
                config_file=config_file,
                early_termination=early_termination,
                render=render,
                history=enable_history,
                **kwargs
            )
            
            # Print summary if requested
            if print_summary:
                print("=== Hierarchy Summary ===")
                example.summary()
                results['summary_printed'] = True
            
            # Get configuration if requested
            if return_config:
                config = example.get_config()
                results['config'] = config
                if verbose:
                    print("=== Configuration Retrieved ===")
            
            # Create hierarchy image if requested
            if create_image:
                if verbose:
                    print("=== Creating Hierarchy Image ===")
                
                # Set default filename if not provided
                if image_params.get('file') is None:
                    import os
                    base_name = os.path.splitext(os.path.basename(config_file))[0]
                    image_params['file'] = f"/tmp/{base_name}_hierarchy.png"
                
                fig = example.draw(**image_params)
                results['image_created'] = True
                results['image_file'] = image_params['file']
                results['figure'] = fig
                
                if verbose:
                    print(f"Hierarchy image saved as: {image_params['file']}")
            
            # Run hierarchy if requested
            if run_hierarchy:
                if verbose:
                    print("=== Running Hierarchy ===")
                
                # Use provided steps or default from config
                run_steps = steps if steps is not None else 1000
                
                run_result = example.run(steps=run_steps, verbose=verbose)
                results['run_result'] = run_result
                results['steps_completed'] = run_steps
                
                if verbose:
                    print(f"Hierarchy completed {run_steps} steps")
            
            # Create video if requested
            if create_video:
                if verbose:
                    print("=== Creating Video ===")
                
                # Set default filename if not provided
                if video_params.get('filename') is None:
                    import os
                    base_name = os.path.splitext(os.path.basename(config_file))[0]
                    video_params['filename'] = f"/tmp/{base_name}_video.mp4"
                
                # Video creation is handled by hierarchy video property, not history
                # This is a placeholder for actual video creation logic
                try:
                    # Video creation would be implemented here using hierarchy video capabilities
                    results['video_created'] = True
                    results['video_file'] = video_params['filename']
                    
                    if verbose:
                        print(f"Video saved as: {video_params['filename']}")
                except Exception as video_error:
                    results['video_error'] = f"Video creation failed: {str(video_error)}"
                    if verbose:
                        print(f"Warning: Video creation failed: {video_error}")
            
            # Create plots if requested
            if create_plots:
                if verbose:
                    print("=== Creating Plots ===")
                
                # Get history data for plotting
                history_data = example.set_history_data()
                if history_data:
                    if plot_params.get('single_plot') and plot_params.get('plots'):
                        # Create single plot
                        plot_fig = example.plot_single(
                            plot=plot_params['plots'],
                            title_prefix=plot_params['title_prefix'],
                            plots_dir=plot_params['plots_dir'],
                            plots_figsize=plot_params['plots_figsize'],
                            history_data=history_data
                        )
                        results['plot_figure'] = plot_fig
                        results['plot_type'] = 'single'
                    else:
                        # Create multiple plots
                        plot_fig = example.plot_history(
                            plots=plot_params['plots'],
                            title_prefix=plot_params['title_prefix'],
                            plots_dir=plot_params['plots_dir'],
                            plots_figsize=plot_params['plots_figsize'],
                            history_data=history_data
                        )
                        results['plot_figure'] = plot_fig
                        results['plot_type'] = 'multiple'
                    
                    results['plots_created'] = True
                    results['history_keys'] = example.get_history_keys()
                    
                    if verbose:
                        print("Plots created successfully")
                        print(f"Available history keys: {results['history_keys']}")
                else:
                    results['plots_error'] = "No history data available for plotting"
                    if verbose:
                        print("Warning: No history data available for plotting")
            
            # Clean up
            example.close()
            results['success'] = True
            
        except Exception as e:
            results['success'] = False
            results['error'] = str(e)
            if verbose:
                print(f"Error: {e}")
        
        return results

    # Usage examples in docstring format
    def _usage_examples():
        """
        Usage Examples:
        
        # Basic run with MountainCar config
        result = PCTExamples.run_example('testfiles/MountainCar/MountainCar-cdf7cc1497ad143c0b04a3d9e72ab783.properties')
        
        # Full featured run
        result = PCTExamples.run_example(
            config_file='testfiles/MountainCar/MountainCar-cdf7cc1497ad143c0b04a3d9e72ab783.properties',
            run_hierarchy=True,
            render=True,
            create_image=True,
            create_video=True,
            create_plots=True,
            early_termination=True,
            steps=5000,
            print_summary=True,
            return_config=True,
            verbose=True,
            image_params={'figsize': (16, 10), 'with_labels': True},
            video_params={'fps': 60, 'filename': '/tmp/mountaincar_demo.mp4'},
            plot_params={'plots_figsize': (14, 8), 'title_prefix': 'MountainCar_'}
        )
        
        # Just create image and get config
        result = PCTExamples.run_example(
            config_file='testfiles/MountainCar/MountainCar-cdf7cc1497ad143c0b04a3d9e72ab783.properties',
            run_hierarchy=False,
            create_image=True,
            create_plots=True,
            return_config=True,
            print_summary=True,
            plot_params={'single_plot': True, 'plots': {'title': 'position_plot', 'plot_items': ['position']}}
        )
        
        # Display usage help
        PCTExamples.run_example('', display_usage=True)
        """
        pass
    

## Separate Video and Plot Creation

The `run_example` class method provides independent control over video and plot creation:

- **`create_video`**: Creates video files using the hierarchy's video property (independent of history)
- **`create_plots`**: Generates matplotlib plots from recorded history data points

These functions are completely separate:

- `video_params`: Controls video creation (fps, filename, etc.) - uses hierarchy video capabilities
- `plot_params`: Controls plot generation (figure size, titles, single vs multiple plots, etc.) - uses history data
- `history`: Records data points specifically for plotting purposes (not needed for video)

Video creation and plot creation use different underlying mechanisms and data sources.

In [ ]:
# Example: Using create_video and create_plots separately

# Define the config file path
config_file = 'testfiles/MountainCar/MountainCar-cdf7cc1497ad143c0b04a3d9e72ab783.properties'

# Create only video (no plots)
video_result = PCTExamples.run_example(
    config_file=config_file,
    run_hierarchy=True,
    steps=1000,
    create_video=True,          # Only video creation
    create_plots=False,         # No plots
    video_params={
        'fps': 30,
        'filename': '/tmp/mountaincar_simulation_video.mp4'
    }
)

# Create only plots (no video)
plots_result = PCTExamples.run_example(
    config_file=config_file,
    run_hierarchy=True,
    steps=1000,
    create_video=False,         # No video
    create_plots=True,          # Only plot creation
    plot_params={
        'plots_figsize': (14, 8),
        'title_prefix': 'MountainCar_',
        'plots_dir': '/tmp/mountaincar_plots'
    }
)

# Use both functions together
combined_result = PCTExamples.run_example(
    config_file=config_file,
    run_hierarchy=True,
    steps=1000,
    create_video=True,          # Create video
    create_plots=True,          # AND create plots
    video_params={'fps': 60, 'filename': '/tmp/mountaincar_combined.mp4'},
    plot_params={'single_plot': True, 'plots': {'title': 'error_analysis', 'plot_items': ['error']}}
)

print("Video only result:", 'video_created' in video_result)
print("Plots only result:", 'plots_created' in plots_result)
print("Combined result:", 'video_created' in combined_result and 'plots_created' in combined_result)

Video only result: False
Plots only result: False
Combined result: False


## History Management

The `run_example` method provides proper history control for plotting:

### History is for Plots Only
- **`history`**: Records data points specifically for plot generation
- **Video creation**: Uses hierarchy video property, completely independent of history
- **Plot creation**: Requires history data to generate matplotlib plots

### Automatic History Detection
- **Default behavior**: History is automatically enabled only when `create_plots=True`
- **Video creation**: Does NOT automatically enable history (uses different mechanism)
- **Plot creation**: Requires history data for chart generation

### Explicit History Control
You can override the automatic behavior using the `history` parameter:

```python
# Force history ON for plots (even if create_plots=False)
result = PCTExamples.run_example('config.json', history=True)

# Force history OFF even with plots (will cause plot errors)
result = PCTExamples.run_example('config.json', create_plots=True, history=False)

# Video without plots (no history needed)
result = PCTExamples.run_example('config.json', create_video=True, create_plots=False)  # history=False (auto)

# Plots without video (history needed)
result = PCTExamples.run_example('config.json', create_video=False, create_plots=True)  # history=True (auto)
```

This ensures `create_video` and `create_plots` use their proper data sources and remain truly independent.

In [ ]:
# Practical Example: MountainCar Environment
# This example demonstrates the complete workflow using the MountainCar test configuration

config_file = 'testfiles/MountainCar/MountainCar-cdf7cc1497ad143c0b04a3d9e72ab783.properties'

# Example 1: Quick hierarchy summary and image
print("=== Example 1: Summary and Hierarchy Visualization ===")
result1 = PCTExamples.run_example(
    config_file=config_file,
    run_hierarchy=False,        # Don't run simulation yet
    create_image=True,          # Create hierarchy diagram
    print_summary=True,         # Print hierarchy info
    return_config=True,         # Get configuration
    verbose=True,
    image_params={
        'figsize': (14, 10),
        'file': '/tmp/mountaincar_hierarchy.png'
    }
)

# Example 2: Run simulation with plots
print("\n=== Example 2: Run Simulation with Plot Generation ===")
result2 = PCTExamples.run_example(
    config_file=config_file,
    run_hierarchy=True,
    steps=500,                  # Run for 500 steps
    create_plots=True,          # Generate plots from history
    verbose=True,
    plot_params={
        'plots_figsize': (12, 8),
        'title_prefix': 'MountainCar_Test_',
        'plots_dir': '/tmp/mountaincar_analysis'
    }
)

# Example 3: Full workflow - simulation, video, and plots
print("\n=== Example 3: Complete Analysis Workflow ===")
result3 = PCTExamples.run_example(
    config_file=config_file,
    run_hierarchy=True,
    render=False,               # No live rendering
    create_video=True,          # Create video file
    create_plots=True,          # Create analysis plots
    steps=1000,                 # Longer simulation
    verbose=True,
    video_params={
        'fps': 30,
        'filename': '/tmp/mountaincar_complete_run.mp4'
    },
    plot_params={
        'plots_figsize': (15, 10),
        'title_prefix': 'Complete_',
        'plots_dir': '/tmp/complete_analysis'
    }
)

print("\n=== Results Summary ===")
print(f"Example 1 - Image created: {result1.get('image_created', False)}")
print(f"Example 2 - Plots created: {result2.get('plots_created', False)}")
print(f"Example 3 - Video created: {result3.get('video_created', False)} | Plots created: {result3.get('plots_created', False)}")

if result2.get('history_keys'):
    print(f"Available plot data: {result2['history_keys']}")

=== Example 1: Summary and Hierarchy Visualization ===
=== Hierarchy Summary ===
**************************
pcthierarchy PCTHierarchy [2, 1] fb00a548-6bc9-11f0-9385-8cf8c5b8669e
--------------------------
PRE: MountainCarContinuousV0 MountainCarContinuousV0 | 0 | links  Action1 
IP IndexedParameter | index 0 | 0.0 | links  MountainCarContinuousV0 
IV IndexedParameter | index 1 | 0.0 | links  MountainCarContinuousV0 
Level 0 Cols 2
L0C0 PCTNode fb00a548-6bc9-11f0-9385-8cf8c5b8669e
----------------------------
REF: RL0C0 SmoothWeightedSum | weights [3.67] smooth 0.17 | -0.0 | links  OL1C0 
PER: PL0C0 SmoothWeightedSum | weights [1] smooth 0.04 | 0.0 | links  IV 
COM: CL0C0 Subtract | -0.0 | links  RL0C0 PL0C0 
OUT: OL0C0 SmoothWeightedSum | weights [-0.68] smooth 0.01 | 0.0 | links  CL0C0 
----------------------------
L0C1 PCTNode fb00a548-6bc9-11f0-9385-8cf8c5b8669e
----------------------------
REF: RL0C1 SmoothWeightedSum | weights [2.24] smooth 0.79 | -0.0 | links  OL1C0 
PER: PL0C1 S

C:\Users\ryoung\Versioning\python\nbdev\pct\pct\hierarchy.py:323: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.



Hierarchy image saved as: mountaincar_hierarchy.png

=== Example 2: Run Simulation with Plot Generation ===
=== Running Hierarchy ===
[0] Error: 'MountainCarContinuousV0' object has no attribute 'actions'

=== Example 3: Complete Analysis Workflow ===
=== Running Hierarchy ===
[0] Error: 'MountainCarContinuousV0' object has no attribute 'actions'

=== Results Summary ===
Example 1 - Image created: True
Example 2 - Plots created: False
Example 3 - Video created: False | Plots created: False


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()